In [1]:
# This notebook includes model developement with target a new target variable as Is_Healthy

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_recall_curve

In [3]:
file_path = 'Data/Cleaned_Impact_of_Remote_Work_on_Mental_Health.csv'
df = pd.read_csv(file_path)

#### Encoding

In [4]:
df['Satisfaction_with_Remote_Work_Encoded'] = df['Satisfaction_with_Remote_Work'].apply(lambda x: 1 if x == 'Satisfied' else 0)

In [5]:
df['Sleep_Quality_Encoded'] = df['Sleep_Quality'].apply(lambda x: 2 if x == 'Good' else (1 if x == 'Average' else 0))

In [6]:
def classify_health_risk(row):
    if row['Stress_Level'] == 'High':
        return 1
    elif row['Stress_Level'] == 'Medium' and (row['Mental_Health_Condition'] != 'Unknown' or row['Sleep_Quality_Encoded'] == 0):
        return 1
    elif row['Stress_Level'] == 'Low' and (row['Mental_Health_Condition'] != 'Unknown' or row['Sleep_Quality_Encoded'] == 0):
        return 1
    else:
        return 0

df['Is_Healthy'] = df.apply(classify_health_risk, axis = 1)

In [7]:
features_to_drop = ['Stress_Level', 'Employee_ID', 'Satisfaction_with_Remote_Work', 'Sleep_Quality', 'Is_Healthy']

In [8]:
X = pd.get_dummies(df.drop(features_to_drop, axis = 1), drop_first = True)
y = df['Is_Healthy']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [10]:
class_distribution = y_train.value_counts()
print(class_distribution)

Is_Healthy
1    3584
0     416
Name: count, dtype: int64


## Using Logistic Regression

#### Tuning

In [ ]:
from sklearn.linear_model import LogisticRegression

param_grid = {
    'C': [0.1, 1, 10, 100],
    'solver': ['liblinear', 'saga']
}

clf = GridSearchCV(LogisticRegression(max_iter = 2000), param_grid, scoring = 'f1', cv = 5)
clf.fit(X_train, y_train)
print("Best Parameters:", clf.best_params_)

#### Train

In [ ]:
from sklearn.linear_model import LogisticRegression

logreg_model = LogisticRegression(
    class_weight = 'balanced', 
    C = clf.best_params_['C'], 
    solver = clf.best_params_['solver'], 
    random_state = 42, 
    max_iter = 1000
)

logreg_model.fit(X_train, y_train)

#### Predict

In [ ]:
y_pred = logreg_model.predict(X_test)

print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

#### Change threshold to improve performance

In [ ]:
y_probs = logreg_model.predict_proba(X_test)[:, 1]
precision, recall, thresholds = precision_recall_curve(y_test, y_probs)
f1_scores = 2 * (precision * recall) / (precision + recall)
best_threshold = thresholds[np.argmax(f1_scores)]

y_pred_adjusted = (y_probs >= best_threshold).astype(int)
print("\nClassification Report:\n", classification_report(y_test, y_pred_adjusted))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_adjusted))